In [2]:
import cv2
import os
import numpy as np
import configparser

# Root directory of the dataset
root_dir = "/home/modesto/PycharmProjects/compressed_tracking/datasets/MOT17"

# Directory where to save the cropped person data
save_dir = "/home/modesto/PycharmProjects/compressed_tracking/datasets/MOT17-reid5D"

# Loop over the sequences
for directory in os.listdir(root_dir):
    # Load the ground truth file
    if not directory.startswith("MOT17"):
        continue

    # This directory is a MOT17 sequence
    sequence = directory

    # Parse the seqinfo.ini file to get original image dimensions
    config = configparser.ConfigParser()
    config.read(os.path.join(root_dir, sequence, "seqinfo.ini"))
    original_width = int(config["Sequence"]["imWidth"])
    original_height = int(config["Sequence"]["imHeight"])

    gt_file = os.path.join(root_dir, sequence, "gt", "gt.txt")
    gt_data = np.loadtxt(gt_file, delimiter=',')

    # Dictionary to hold person data
    person_data = {}

    # Loop over the ground truth data
    for frame_num, person_id, x, y, w, h, _, _, _ in gt_data:
        # Convert frame_num and person_id to integers
        frame_num = int(frame_num)
        person_id = int(person_id)
        
        if frame_num % 12 == 1:
            continue

        # Load the image file for this frame
        img_file = os.path.join(root_dir, sequence, "residual", f"{frame_num:06}.jpg")
        try:
            img = cv2.imread(img_file)
        except:
            continue  # skip this frame if the image file does not exist

        # Handle case where image is None
        if img is None:
            continue
     #   print("frame_num",frame_num)
     #   print(f"Original bounding box values: x={x}, y={y}, w={w}, h={h}")

        # Get the actual dimensions of the image
        actual_height, actual_width, _ = img.shape

        # Compute scale factors
        scale_y = actual_height / original_height
        scale_x = actual_width / original_width

        # Scale the bounding box coordinates
        x, w = int(x * scale_x), int(w * scale_x)
        y, h = int(y * scale_y), int(h * scale_y)
        
      #  print(f"Scaled bounding box values: x={x}, y={y}, w={w}, h={h}")

        # Crop the image
        x = max(0, x)
        y = max(0, y)
        w = min(w, actual_width - x)
        h = min(h, actual_height - y)

        # If the bounding box has become zero-sized due to corrections, skip it
        if w <= 0 or h <= 0:
            continue

        cropped_img = img[y:y+h, x:x+w]

        # Handle case where cropped_img is empty
        if cropped_img.size == 0:
            print(f"Empty cropped image for frame: {frame_num}{person_id}")
            print(f"Bounding box coordinates: x={x}, y={y}, w={w}, h={h}")
            print(f"Original image size: {img.shape}")

            continue

        # Create a directory to hold this person's data if it doesn't exist yet
        if person_id not in person_data:
            person_data[person_id] = os.path.join(save_dir, sequence, str(person_id))
            os.makedirs(person_data[person_id], exist_ok=True)

        # Save the cropped image
        cv2.imwrite(os.path.join(person_data[person_id], f"{frame_num:06}.jpg"), cropped_img)

        # Load the data file for this frame
        data_file = os.path.join(root_dir, sequence, "mv", f"{frame_num:06}.npy")
        try:
            data = np.load(data_file)
        except FileNotFoundError:
            continue  # skip this frame if the .npy file does not exist

        # Crop the data
        cropped_data = data[y:y+h, x:x+w, :]

        # Save the cropped data
        np.save(os.path.join(person_data[person_id], f"{frame_num:06}.npy"), cropped_data)



In [3]:
import os
import shutil

# Le chemin vers votre répertoire principal et le répertoire de destination
main_dir = 'test_dataset'
dest_dir = 'deep_sort_dataset'


os.makedirs(dest_dir, exist_ok=True)

# Parcourir tous les répertoires dans le répertoire principal
for dir_name in os.listdir(main_dir):
    dir_path = os.path.join(main_dir, dir_name)
    
    # Vérifier si c'est un répertoire
    if os.path.isdir(dir_path):
        # Extraire le numéro de séquence du nom du répertoire
        sequence_number = dir_name.split('-')[1]
        
        # Parcourir tous les sous-répertoires dans le répertoire courant
        for subdir_name in os.listdir(dir_path):
            subdir_path = os.path.join(dir_path, subdir_name)
            
            # Vérifier si c'est un répertoire
            if os.path.isdir(subdir_path):
                # Créer un nouveau nom de sous-répertoire préfixé par le numéro de séquence
                new_subdir_name = f"{sequence_number}_{subdir_name}"
                new_subdir_path = os.path.join(dest_dir, new_subdir_name)
                
                # Créer le nouveau sous-répertoire dans le répertoire de destination
                os.makedirs(new_subdir_path, exist_ok=True)
                
                # Parcourir tous les fichiers dans le sous-répertoire courant
                for file_name in os.listdir(subdir_path):
                    file_path = os.path.join(subdir_path, file_name)
                    
                    # Ignore if it's a directory
                    if os.path.isdir(file_path):
                        continue
                    
                    # Créer le chemin vers le nouveau fichier
                    new_file_path = os.path.join(new_subdir_path, file_name)
                    
                    # Copier le fichier dans le nouveau sous-répertoire
                    shutil.copy(file_path, new_file_path)


In [7]:
import cv2
import numpy as np

PIXEL_MEANS = np.array([[[0.485, 0.456, 0.406]]]) # BGR
PIXEL_STDS = np.array([[[0.229, 0.224, 0.225]]]) # BGR

#mean: [0.485, 0.456, 0.406]
#std: [0.229, 0.224, 0.225]

def denormalize_image(image):
    image = image.astype(np.float32) 
    image *= PIXEL_STDS
    image += PIXEL_MEANS
    return image.astype(np.uint8)

image_path =  "/home/jovyan/Desktop/MOT17_residual/train/MOT17-04-PRIVATE/img1/001049.jpg"
image = cv2.imread(image_path)

# Dé-normaliser l'image
image = denormalize_image(image)

# Convertir de BGR en RGB
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

cv2.imwrite('/home/jovyan/Desktop/image.jpg', image)


True